(C:deconvolution)=
# Deconvolution

La déconvolution consiste à annuler l'effet de la convolution par un filtre.
Il est fréquent, en effet, que l'acquisition d'une image introduise une dégradation de celle-ci qui peut être modélisée par une convolution.
C'est le cas par exemple d'un effet de bougé ({numref}`F:deconvolution:example-motion`)
ou de flou dû à une mauvaise mise au point ({numref}`F:deconvolution:example-blur`).

```{figure} figs/restauration9.png
---
width: 600px
name: F:deconvolution:example-motion
---
Flou de bougé et le résultat d'une déconvolution idéale.
```

```{figure} figs/restauration10.png
---
width: 600px
name: F:deconvolution:example-blur
---
Flou de mise au point et le résultat d'une déconvolution idéale.
```

Le problème est donc modélisé comme dans la {numref}`F:deconvolution:model` :
l'image observée $y$ est dégradée par la convolution avec une PSF $h$ et, éventuellement, par un bruit $b$ (que l'on considère additif).
Le modèle de la dégradation est donc :

$$
y = h*x + b
$$

À partir de l'observation $y$, la déconvolution calcule une image déconvoluée $\widehat{x}$.
Dans le cadre du cours, nous ne considérons que les méthodes linéaires, qui reviennent donc à un filtrage par $g$ :

$$
\widehat{x} = g*y
$$

```{figure} figs/restauration11.png
---
width: 400px
name: F:deconvolution:model
---
Modèle de la déconvolution
```

Pour effectuer au mieux la déconvolution, il faut donc un modèle de la dégradation
(donc connaître à la fois $h$ et $b$).

La PSF $h$ peut être estimée par observation, c'est-à-dire en trouvant dans l'image des indices permettant d'estimer $h$
(par exemple, un objet ponctuel isolé dans l'image deviendra réellement $h$).
Elle peut également être estimée par expérimentation, donc en reproduisant en laboratoire les conditions d'observation.
L'image d'une impulsion observée ainsi donne une estimation de $h$.
Enfin, il est possible d'estimer la PSF en modélisant mathématiquement la physique de l'observation.
Notons par ailleurs que certaines méthodes de déconvolution estiment la PSF $h$ en même temps que $x$ :
ce sont des méthodes de déconvolution myope (_blind deconvolution_).
<!-- développer le 1er exemple, notamment avec une illustration en astro -->
<!-- 1er point : déconvolution naïve en l'absence de bruit dans une petite zone, essai--erreur, ..-->

Les modèles de bruit ont déjà été présentés dans le chapitre {ref}`C:denoising`.

## Filtre inverse

Le filtre inverse est la méthode de déconvolution la plus simple.
Puisque la dégradation se modélise $y = h*x + b$, alors dans le domaine de Fourier cette équation devient :

$$
Y = HX + B
$$

et on peut donc écrire :

$$
X = \frac{Y-B}{H}.
$$

On obtient donc $x$ en calculant la transformée de Fourier inverse de l'expression précédente :

$$
x = \mathcal{F}^{-1} \left[ \frac{Y-B}{H} \right].
$$

Comme le bruit (et donc son spectre $B$) est inconnu, on peut approximer l'expression de $x$ en annulant $B$ dans l'expression précédente,
et donc obtenir l'image déconvoluée :

$$
\widehat{x} = \mathcal{F}^{-1} \left[ \frac{Y}{H} \right]
$$

Le résultat du filtre inverse est donné {numref}`F:deconvolution:inverse`.
Le résultat n'est absolument pas exploitable, et pourtant l'image observée est très peu floutée et très peu bruitée !

```{figure} figs/restauration12.png
---
width: 800px
name: F:deconvolution:inverse
---
Résultat du filtre inverse.
De gauche à droite : image originale $x$, image observée $y$, sortie du filtre inverse $\widehat{x}$,
sortie du filtre inverse tronqué $\widehat{x}$ (10 % de la taille de l'image).
```

Le résultat catastrophique du filtre inverse est dû au fait d'avoir considéré le bruit nul.
En effet, d'après la définition de $\widehat{x}$, et puisque $Y=HX+B$, alors :

$$
\widehat{x}
= \mathcal{F}^{-1} \left[ \frac{Y}{H} \right]
= \mathcal{F}^{-1} \left[ X + \frac{B}{H} \right]
= x + \mathcal{F}^{-1} \left[ \frac{B}{H} \right]
$$

Ainsi, l'image déconvoluée $\widehat{x}$ correspond à $x$ avec un terme parasite qui est la transformée de Fourier inverse de $B/H$.
Or, très souvent, la PSF $H$ est un filtre passe-bas,
ce qui implique que les valeurs de $H(m,n)$ tendent vers $0$ lorsque les fréquences $(m,n)$ deviennent grandes.
$H$ étant au dénominateur du terme parasite, elle a tendance à amplifier les hautes fréquences du bruit d'un facteur qui tend vers $+\infty$.
Finalement, le terme $B/H$ peut rapidement dominer $X$.
Cela explique le résultat de la {numref}`F:deconvolution:inverse`.

Une solution consiste à calculer non pas la transformée de Fourier inverse de toute l'image $Y/H$,
mais seulement de ses basses fréquences.
Cela signifie que les hautes fréquences de $Y/H$, qui peuvent tendre vers l'infini comme on l'a dit,
sont annulées avant de calculer la transformée de Fourier inverse.
Autrement dit, le filtre inverse est tronqué à une partie du spectre.
Ainsi, le résultat de la déconvolution est beaucoup plus acceptable, comme le montre la {numref}`F:deconvolution:inverse`,
bien que le résultat soit encore loin d'être parfait
(il y a de nombreuses variations d'intensité autour des objets, comme les tronc des arbres)...

## Filtre de Wiener

Le filtre de Wiener, noté $g$ (et sa transformée de Fourier $G$), s'applique sur l'observation $y$ :

$$
y = g * y
\qquad\Leftrightarrow\qquad
Y = GY.
$$

Ce filtre est établi dans le cadre statistique : l'image $x$ et le bruit $b$ sont considérés comme étant des variables aléatoires.
Elles sont également supposées statistiquement indépendantes.
Il en résulte que l'observation $y$ et l'estimation $\widehat{x}$ sont également des variables aléatoires.

Les calculs sont effectués dans le domaine de Fourier, pour utiliser des multiplications plutôt que des convolutions.
L'objectif du filtre de Wiener est donc de trouver l'image $\widehat{X} = \mathcal{F}[\widehat{x}]$ la plus proche de $X=\mathcal{F}[x]$,
au sens de l'erreur quadratique moyenne $\mathrm{EQM} = \mathbb{E}\left[(\widehat{X}-X)^2\right]$.
Ainsi :

$$
\mathrm{EQM}
&= \mathbb{E}\left[(\widehat{X}-X)^2\right] \\
&= \mathbb{E}\left[(GY-X)^2\right] \\
&= \mathbb{E}\left[\big(G(HX+B)-X\big)^2\right] \\
&= \mathbb{E}\left[\big((GH-I)X+GB\big)^2\right]
$$

où $I$ est une image remplie de 1.
En développant le carré (et en notant le conjugué des variables avec $\cdot^*$), on obtient :

$$
\mathrm{EQM} = \mathbb{E}\Big[ (GH-I)^*(GH-I)X^*X + (GH-I)^*GX^*B + G^*(GH-I)B^*X + G^*GB^*B \Big].
$$

Comme l'espérance est linéaire et que seules $X$ et $B$ sont des variables aléatoires,
on peut décomposer l'expression précédente en quatre termes :


$$
\mathrm{EQM}
= (GH-I)^*(GH-I) \mathbb{E}\big[X^*X\big]
+ (GH-I)^*G \mathbb{E}\big[X^*B\big]
+ G^*(GH-I) \mathbb{E}\big[B^*X\big]
+ G^*G \mathbb{E}\big[B^*B\big].
$$

Comme $X$ et $B$ sont indépendantes, alors leurs covariances $\mathbb{E}\big[X^*B\big]$ et $\mathbb{E}\big[B^*X\big]$ sont nulles.
Par ailleurs, $\mathbb{E}\big[X^*X\big]$ et $\mathbb{E}\big[B^*B\big]$ sont les densités spectrales de puissances notées $S_x$ et $S_b$
(on rappelle que la densité spectrale de puissance est l'espérance du carré du module de la transformée de Fourier).
Donc l'erreur quadratique se simplifie en :

$$
\mathrm{EQM} = (GH-1)^*(GH-1) S_x + G^*G S_b
$$

Il faut donc chercher le filtre $G$ qui minimise l'EQM, ou demanière équivalente, qui annule la dérivée de l'EQM :

$$
                       & \frac{\partial \mathrm{EQM}}{\partial G} = (GH-1)^*H S_x + G^* S_b = 0 \\
  \Leftrightarrow\quad &G^*H^*H S_x - H S_x + G^* S_b = 0  \\
  \Leftrightarrow\quad &G^* ( H^*H S_x + S_b) = H S_x  \\
  \Leftrightarrow\quad &G^* = \frac{H S_x}{H^*H S_x + S_b}  \\
  \Leftrightarrow\quad &G = \frac{H^* S_x}{H^*H S_x + S_b}  \\
  \Leftrightarrow\quad &G = \frac{H^* S_x}{|H|^2 S_x + S_b}
$$

On a obtenu l'expression de $G$ qui constitue le filtre de Wiener !
Finalement, l'image déconvoluée est la transformée de Fourier inverse de $GY$ :

$$
\widehat{x} = \mathcal{F}^{-1} \Bigg[\frac{H^* S_x}{|H|^2 S_x + S_b} Y \Bigg]
$$

On peut considérer que les densités spectrales de puissance $S_x$ et $S_b$ sont constantes
(pour $S_b$, il faut faire l'hypothèse d'un bruit blanc).
Ainsi, l'expression du filtre de Wiener peut s'écrire

$$
\widehat{x} = \mathcal{F}^{-1} \Bigg[ \frac{H^*}{|H|^2 + S_b/S_x} Y \Bigg]
$$

et le terme $S_b/S_x$ est remplacé par une constante $K$, qui devient le paramètre de la méthode, à régler par l'utilisateur.

Remarques :

* On note que lorsque $H$ s'annule (typiquement dans les hautes fréquences),
  on n'observe plus le problème d'augmentation du bruit comme avec le filtre inverse,
  puisque le filtre inverse tend vers 0.

* Par ailleurs, si le bruit dans l'image est nul, alors $S_b = 0$ et on obtient le filtre inverse :
  
  $$
  \widehat{x}
  = \mathcal{F}^{-1} \Bigg[ \frac{H^*}{|H|^2} Y \Bigg]
  = \mathcal{F}^{-1} \Bigg[ \frac{Y}{H} \Bigg]
  $$
  
Le résultat du filtre de Wiener est présenté {numref}`F:deconvolution:wiener` :
le résultat est clairement bien meilleur que le filtre de Wiener, même tronqué !

```{figure} figs/restauration14.png
---
width: 600px
name: F:deconvolution:wiener
---
Résultat du filtre de Wiener, pour $K=10^{-5}$,
comparé au filtre inverse tronquée (10 % de la taille de l'image).
```

Cela dit, il n'est pas juste de comparer ces deux méthodes sur la {numref}`F:deconvolution:wiener`,
car les paramètres des méthodes (10% et $K=10^{-5}$) ne sont peut être pas les meilleurs.
Il se pose donc la question : comment régler le paramètre de chaque méthode ?

```{margin}
L'EQM dont il est question ici n'est pas au sens statistique, comme ci-avant.
Il s'agit ici du carré de la différence entre les deux images.
```

Pour y répondre, la {numref}`F:deconvolution:parametre` représente la qualité du résultat, en termes d'EQM,
en fonction de ces paramètres.
On constate que pour le meilleur choix des paramètres (point rouge sur la courbe),
l'EQM est bien meilleure pour le filtre de Wiener que pour le filtre inverse.
Le résultat correspondant est représenté {numref}`F:deconvolution:wiener-best`.
La différence sur cette sous-image est assez faible, mais il n'empêche que le dessin sur la voile est plus net avec le filtre de Wiener.

```{figure} figs/restauration15.png
---
width: 600px
name: F:deconvolution:parametre
---
Comparaison de l'EQM en fonction du paramètre de la méthode, pour le filtre inverse tronqué et le filtre de Wiener.
```

```{figure} figs/restauration16.png
---
width: 600px
name: F:deconvolution:wiener-best
---
Résultat du filtre inverse tronquée et du filtre de Wiener pour les meilleurs choix de paramètres.
```

## Conclusion

En résumé, la déconvolution consiste à inverser le filtrage (généralement passe-bas) qu'a subie l'image observée.
Le filtre inverse ne fonctionne pas car il tend à augmenter le bruit présent dans l'image.
Le filtre de Wiener reste donc la méthode de déconvolution la plus simple et qui donne un résultat acceptable.
De nombreuses autres méthodes ont été développées, comme la méthode de Richardson-Lucy [Richardson 1972, Lucy 1974] ou les méthodes bayésiennes.

<!-- MAP Contrained least squares -->